---

### <font color="red">config the network</font>

In [1]:
from IoT.neuron import * 
from time import sleep
import pandas as pd
from pandas import DataFrame

pd.options.display.max_colwidth = 400
REFRACTORY_PERIOD = 0.1   # 0.1 seconds

In [2]:
# there are 6 neurons，each is a Docker container，and was deployed on a Docker Swarm machine.
neurons = ['neuron_x', 'neuron_y', 'neuron_h1', 'neuron_h2', 'neuron_h3', 'neuron_z'] 

# print log from a neuron
def printConfig(neuron):
    print('{0:_^78}\n {1}\n'.format(neuron + " config:", getConfig.apply_async(routing_key = neuron).get()))

# clear log in a neuron  
def emptyLogs():
    for neuron in neurons:
        emptyLog.apply_async(routing_key = neuron)

# collect logs。concate logs from all neurons into a Pandas.DataFrame
def mergeLogs():
    logs = []
    
    for neuron in neurons:
        currentLog = getLog.apply_async(routing_key = neuron).get()
        logs += currentLog 
            
    df = DataFrame(list(logs), columns = ['time', 'neuron', 'message']) 
    df.set_index('time', inplace = True)
    df.sort_index(inplace = True)
    
    return df

### clear log files

In [3]:
# clear logs from all neurons.
emptyLogs()

### setup connections

In [4]:
# input layer fan out
# neuron x
addConnection.apply_async(['neuron_h1'], routing_key = 'neuron_x')  # 增設 neuron_x -> neuron_h1 的 connection
addConnection.apply_async(['neuron_h2'], routing_key = 'neuron_x')
# neuron y
addConnection.apply_async(['neuron_h2'], routing_key = 'neuron_y')  # 增設 neuron_y -> neuron_h2 的 connection
addConnection.apply_async(['neuron_h3'], routing_key = 'neuron_y')

# hidden layer fan out
addConnection.apply_async(['neuron_z'], routing_key = 'neuron_h1')  # 增設 neuron_h1 -> neuron_z 的 connection
addConnection.apply_async(['neuron_z'], routing_key = 'neuron_h2')
addConnection.apply_async(['neuron_z'], routing_key = 'neuron_h3')

<AsyncResult: 01754554-a45b-48ee-bb1d-72bfb7428c79>

### setup weights

In [5]:
# hidden layer
setWeight.apply_async(['neuron_x', 1], routing_key = 'neuron_h1')  # 設定 neuron_x -> neuron_h1 的 weight = 1
setWeight.apply_async(['neuron_x', 1], routing_key = 'neuron_h2')
setWeight.apply_async(['neuron_y', 1], routing_key = 'neuron_h2')  # 設定 neuron_y -> neuron_h2 的 weight = 1
setWeight.apply_async(['neuron_y', 1], routing_key = 'neuron_h3')

# output layer
setWeight.apply_async(['neuron_h1', 1], routing_key = 'neuron_z')
setWeight.apply_async(['neuron_h2', -2], routing_key = 'neuron_z')  # 設定 neuron_h2 -> neuron_z 的 weight = -2 (inhibitory)
setWeight.apply_async(['neuron_h3', 1], routing_key = 'neuron_z') 

<AsyncResult: d1461e9e-1d44-4713-be90-465ac93b453a>

### setup thresholds

In [6]:
# input layer 
setThreshold.apply_async([0.9], routing_key = 'neuron_x')  # 設定 neuron_x 的 threshold = 0.9
setThreshold.apply_async([0.9], routing_key = 'neuron_y') 

# hidden layer
setThreshold.apply_async([0.9], routing_key = 'neuron_h1') 
setThreshold.apply_async([1.9], routing_key = 'neuron_h2')  # 設定 neuron_h2 的 threshold = 1.9
setThreshold.apply_async([0.9], routing_key = 'neuron_h3')

# output layer
setThreshold.apply_async([0.9], routing_key = 'neuron_z')  # 設定 neuron_z 的 threshold = 0.9

<AsyncResult: a3be2285-1449-4b93-8922-0720baa3b5f2>

### simulate sensor inputs，and check status of neurons.

In [7]:
emptyLogs()  # clear logs
sleep(REFRACTORY_PERIOD)  # wait for stead state
mergeLogs()  # summarize logs

,neuron,message
time,,


In [8]:
### force neuron x to ouput 1
emptyLogs()  # clear logs
sleep(REFRACTORY_PERIOD)  # wait for stead state
fire.apply_async(routing_key = 'neuron_x') # force neuron x output 1 and fire.
mergeLogs()  # summarize logs

,neuron,message
time,,
2016-12-22 19:20:43.784441,neuron_x,neuron_x fires.
2016-12-22 19:20:43.788134,neuron_x,Setting output of neuron_x to ACTION_POTENTIAL.
2016-12-22 19:20:43.905049,neuron_h2,neuron_x is kicking neuron_h2.
2016-12-22 19:20:43.948279,neuron_h1,neuron_x is kicking neuron_h1.
2016-12-22 19:20:43.962901,neuron_h1,neuron_h1 fires.
2016-12-22 19:20:43.966272,neuron_h1,Setting output of neuron_h1 to ACTION_POTENTIAL.
2016-12-22 19:20:44.027375,neuron_z,neuron_h1 is kicking neuron_z.
2016-12-22 19:20:44.048686,neuron_z,neuron_z fires.
2016-12-22 19:20:44.052765,neuron_z,Setting output of neuron_z to ACTION_POTENTIAL.


In [9]:
### force neuron y to ouput 1
emptyLogs()  # clear logs
sleep(REFRACTORY_PERIOD)  # wait for stead state.
fire.apply_async(routing_key = 'neuron_y') # force neuron y output 1 and fire.
mergeLogs()  # summarize logs

,neuron,message
time,,
2016-12-22 19:20:47.221569,neuron_y,neuron_y fires.
2016-12-22 19:20:47.225052,neuron_y,Setting output of neuron_y to ACTION_POTENTIAL.
2016-12-22 19:20:47.333917,neuron_h2,neuron_y is kicking neuron_h2.
2016-12-22 19:20:47.367095,neuron_h3,neuron_y is kicking neuron_h3.
2016-12-22 19:20:47.380941,neuron_h3,neuron_h3 fires.
2016-12-22 19:20:47.383612,neuron_h3,Setting output of neuron_h3 to ACTION_POTENTIAL.
2016-12-22 19:20:47.444634,neuron_z,neuron_h3 is kicking neuron_z.
2016-12-22 19:20:47.460059,neuron_z,neuron_z fires.
2016-12-22 19:20:47.463746,neuron_z,Setting output of neuron_z to ACTION_POTENTIAL.


In [10]:
### force neuron x and y to ouput 1
emptyLogs()  # clear logs
sleep(REFRACTORY_PERIOD)  # wait for stead state.
fire.apply_async(routing_key = 'neuron_x') # force neuron x output 1 and fire.
fire.apply_async(routing_key = 'neuron_y') # force neuron y output 1 and fire.
mergeLogs()  # summarize logs

,neuron,message
time,,
2016-12-22 19:20:50.764877,neuron_x,neuron_x fires.
2016-12-22 19:20:50.769051,neuron_x,Setting output of neuron_x to ACTION_POTENTIAL.
2016-12-22 19:20:50.786583,neuron_y,neuron_y fires.
2016-12-22 19:20:50.790631,neuron_y,Setting output of neuron_y to ACTION_POTENTIAL.
2016-12-22 19:20:50.887841,neuron_h2,neuron_x is kicking neuron_h2.
2016-12-22 19:20:50.907530,neuron_h1,neuron_x is kicking neuron_h1.
2016-12-22 19:20:50.930401,neuron_h1,neuron_h1 fires.
2016-12-22 19:20:50.933333,neuron_h1,Setting output of neuron_h1 to ACTION_POTENTIAL.
2016-12-22 19:20:50.956406,neuron_h3,neuron_y is kicking neuron_h3.


### [Flower](http://192.168.0.114:5555) shows the status of each worker :

![各 neuron 的活動次數](https://github.com/Wei1234c/IOTasBrain/raw/master/celery_projects/jpgs/flower2.jpg "各 neuron 的活動次數")

### content of config files in each neurons:

In [11]:
for neuron in reversed(neurons): printConfig(neuron)

_______________________________neuron_z config:_______________________________
 {'inputs': {'neuron_h2': {'kick_time': datetime.datetime(2016, 12, 22, 19, 20, 51, 125376), 'lasting': datetime.timedelta(0, 0, 500000), 'value': 1}, 'neuron_h3': {'kick_time': datetime.datetime(2016, 12, 22, 19, 20, 51, 74644), 'lasting': datetime.timedelta(0, 0, 500000), 'value': 1}, 'neuron_h1': {'kick_time': datetime.datetime(2016, 12, 22, 19, 20, 51, 2932), 'lasting': datetime.timedelta(0, 0, 500000), 'value': 1}}, 'weights': {'neuron_h2': -2, 'neuron_h3': 1, 'neuron_h1': 1}, 'output': {'value': 1, 'lasting': datetime.timedelta(0, 0, 100000), 'polarized_time': datetime.datetime(2016, 12, 22, 19, 20, 51, 27395)}, 'threshold': 0.9}

______________________________neuron_h3 config:_______________________________
 {'inputs': {'neuron_y': {'kick_time': datetime.datetime(2016, 12, 22, 19, 20, 50, 966575), 'lasting': datetime.timedelta(0, 0, 500000), 'value': 1}}, 'weights': {'neuron_y': 1}, 'connections': {'n